# Importando Biblioteca de Funções Turing Lab e Turing IA

In [1]:
%run -i '/home/hadoop/.Turing/TuringAllFunctions.py'

------------- Turing Lab Data Science --------------------
------ Funções carregadas em memória com sucesso ---------
--------- Data da última atualização: --------------------
--------------- 03/07/2019 -------------------------------
----------------------------------------------------------
--- Consulte o manual de Biblioteca de Funções Turing: ---


----------------------------------------------------------


# Selecionando Projeto Desenvolvido no Turing IA 

In [4]:
# id_empresa = idempresa.value
# id_projeto = idprojeto.value
# nm_file = nm_abt.value
# targetname = target.value
# abt_delimiter = delimit.value
# nm_bucket = nmbucket.value

#---------- Definição dos Parâmetros do projeto ---------------------------------
nm_bucket = 'turing-bkt-powerofdata'
id_projeto = '1000'
nm_file = '05_train.csv'
id_empresa = '1000' 
targetname = 'Survived'
abt_delimiter = ','

In [5]:
#---------- Gerando Credenciais para Acesso Seguro -------------------------------
id_modelo = id_projeto
schema = 'EMP_'+ id_empresa +'_Proj_'+ id_projeto
S3fs,S3session,S3client,S3resource = TuringUsersCredentialsControl(id_empresa)
schema_rds = 'EMP_'+id_empresa+'_Proj_'+id_projeto
mysq_conn,engine = TuringRDSManagementConn(id_empresa,schema_rds,turing_adm = 'N')
path_s3_wfile = 's3://'+nm_bucket+'/Projetos/'+'Projeto_'+id_projeto+'/Modelo_Id_'+id_projeto+'/Modeltrain/'
s3_path = path_s3_wfile

# Importando dados (do S3)

In [5]:
# Lendo arquivo do S3 ------------------------------------------------------
path_file = 'Projetos/FLAT_TABLES/'+str(nm_file)
train_df = TuringReadS3File(S3client,nm_bucket,path_file,sep=abt_delimiter)

In [6]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Objetivo: Gerar novas variáveis para melhorar o modelo

In [7]:
# Vamos extrair o titulo do Nome (Mr., Miss...)
train_df['Titulo'] = train_df.Name.str.extract('([A-Za-z]+)\.', expand=False)

train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Titulo
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,Mr
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,Mr
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,Master
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,Mrs
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,Mrs


In [8]:
# Verificando Taxa de Evento por Titulo
df_titulo = TuringTxEventoCateg(train_df,'Titulo','Survived')
df_titulo

,Titulo,Tx_Evento,Volume
16,Sir,1.000000,1
2,Countess,1.000000,1
14,Ms,1.000000,1
11,Mme,1.000000,1
6,Lady,1.000000,1
10,Mlle,1.000000,2
13,Mrs,0.792000,125
9,Miss,0.697802,182
8,Master,0.575000,40
1,Col,0.500000,2


In [9]:
# Com o resultado anterior vimos que podemos agrupar/substituir alguns títulos

# Substituindo lista de títulos com menor frequencia por Raros
list_dominios = ['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona']
train_df = TuringAgrupCategs(train_df,'Titulo',list_dominios,'Raros')

# Substituindo Mlle por Miss
train_df = TuringAgrupCategs(train_df,'Titulo','Mlle', 'Miss')

# Substituindo Ms por Miss
train_df = TuringAgrupCategs(train_df,'Titulo','Ms', 'Miss')

# Substituindo Mme por Mrs
train_df = TuringAgrupCategs(train_df,'Titulo','Mme', 'Mrs')


In [10]:
# Verificando Novamente Taxa de Evento por Titulo (Agora tratado)
df_titulo = TuringTxEventoCateg(train_df,'Titulo','Survived')
df_titulo

,Titulo,Tx_Evento,Volume
3,Mrs,0.793651,126
1,Miss,0.702703,185
0,Master,0.575000,40
4,Raros,0.347826,23
2,Mr,0.156673,517


In [11]:
# Vamos verificar a variável nova que foi adicionada à tabela de modelagem
train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Titulo
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,Mr
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,Mr
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,Master
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,Mrs
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,Mrs


In [12]:
# Como criamos a variável nome, vamos deletar a variável Name
train_df = TuringDropVars(train_df,'Name')

In [13]:
# Verificando como ficou a nova tabela de modelagem
train_df.head(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Titulo
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S,Mr
5,6,0,3,male,NaN,0,0,330877,8.4583,NaN,Q,Mr
6,7,0,1,male,54.0,0,0,17463,51.8625,E46,S,Mr
7,8,0,3,male,2.0,3,1,349909,21.0750,NaN,S,Master
8,9,1,3,female,27.0,0,2,347742,11.1333,NaN,S,Mrs
9,10,1,2,female,14.0,1,0,237736,30.0708,NaN,C,Mrs


In [14]:
# Salvar nova tabela no S3 para refazer o modelo com o Turing IA
path_s3_wfile = 's3://'+nm_bucket+'/Projetos/'+'FLAT_TABLES/'
nm_s3_file = 'teste_torq.csv'
TuringWriteS3File(train_df,path_s3_wfile,nm_s3_file,S3fs,delimiter=',')

'written successfully'

# Resultado da primeira variável construída (text mining)

In [15]:
# Resultado:
# Gini (Modelo Original)           ---> 61.9
# Gini (Modelo c/ Nova Variável)   ---> 72.88

# Segunda rodada de criação de variáveis

In [16]:
# Vamos criar mais uma variável para ver se o desempenho melhora
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1

In [17]:
train_df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Titulo,FamilySize
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,2
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,2
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,1
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,Mrs,2
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S,Mr,1


In [18]:
# Verificando Novamente Taxa de Evento por Titulo (Agora tratado)
df_fs = TuringTxEventoCateg(train_df,'FamilySize','Survived')
df_fs

,FamilySize,Tx_Evento,Volume
3,4,0.724138,29
2,3,0.578431,102
1,2,0.552795,161
6,7,0.333333,12
0,1,0.303538,537
4,5,0.200000,15
5,6,0.136364,22
7,8,0.000000,6
8,11,0.000000,7


In [19]:
# Criando flag que identifica se a pessoa está sozinha no Navio
train_df['IsAlone'] = 0
train_df.loc[train_df['FamilySize'] == 1, 'IsAlone'] = 1

In [20]:
# Verificando Novamente Taxa de Evento 
df_ia = TuringTxEventoCateg(train_df,'IsAlone','Survived')
df_ia

,IsAlone,Tx_Evento,Volume
0,0,0.505650,354
1,1,0.303538,537


In [21]:
# Vamos gerar nova tabela de modelagem
# Salvar nova tabela no S3 para refazer o modelo com o Turing IA
path_s3_wfile = 's3://'+nm_bucket+'/Projetos/'+'FLAT_TABLES/'
nm_s3_file = 'titanic_train_abt02.csv'
TuringWriteS3File(train_df,path_s3_wfile,nm_s3_file,S3fs,delimiter=',')

'written successfully'

# Novo resultado

In [ ]:
# Resultado:
# Gini (Modelo Original)           ---> 61.9
# Gini (Modelo c/ Nova Variável)   ---> 72.88
# Gini (Modelo c/ Nova Variável)   ---> 74.14